In [1]:
import os    
#os.environ['THEANO_FLAGS'] = "device=gpu1"  
#os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu0,floatX=float32,lib.cnmem=1"
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu0,floatX=float32"
import theano


import numpy as np
np.random.seed(1337) # for reproducibility
from sklearn.cross_validation import train_test_split

from keras.datasets import mnist
from keras.models import Sequential
#from keras.layers import containers
from keras.layers import Input, Dense
#from keras.layers.core import AutoEncoder
from keras.activations import sigmoid
from keras.utils import np_utils

from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import *

nb_classes = 10
nb_hidden_layers = [784, 676, 484, 324, 144, 64, 35]

Using gpu device 0: GeForce 920M (CNMeM is disabled, cuDNN not available)
Using Theano backend.


In [2]:
# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0
print(X_train.shape, 'train samples')
print(X_test.shape, 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)



(60000, 784) train samples
(10000, 784) test samples


In [3]:
# Layer-wise pretraining
encoders = []
X_train_tmp = np.copy(X_train)
print(X_train_tmp.shape)
nfold=10
nb_epoch=50
batch_size=256
random_state =6567

(60000, 784)


In [4]:
for i, (n_in, n_out) in enumerate(zip(nb_hidden_layers[:-1], nb_hidden_layers[1:]), start=1):
    print('Training the layer {}: Input {} -> Output {}'.format(i, n_in, n_out))
    
    # this is our input placeholder
    input_img = Input(shape=(n_in,))
    # "encoded" is the encoded representation of the input
    encoded = Dense(n_out, activation='relu')(input_img)
    # "decoded" is the lossy reconstruction of the input
    decoded = Dense(n_in, activation='relu')(encoded)

    # this model maps an input to its reconstruction
    # we will not need a model to reconstruct
    autoencoder = Model(input=input_img, output=decoded)
    
    # this model maps an input to its encoded representation
    # this is the important model for encoding and need to be stacked
    encoder = Model(input=input_img, output=encoded)

    # create a placeholder for an encoded  input
    encoded_input = Input(shape=(n_out,))
    # retrieve the last layer of the autoencoder model
    decoder_layer = autoencoder.layers[-1]
    # create the decoder model
    decoder = Model(input=encoded_input, output=decoder_layer(encoded_input))

    print('compiling...')
    autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
    print('fiting.pretraining for just few epochs..')
    
    autoencoder.fit(X_train_tmp, X_train_tmp,
                nb_epoch=1,
                batch_size=256,
                shuffle=True)
    
    # encode and decode some digits
    # note that we take them from the *test* set
    #encoded_imgs = encoder.predict(X_train_tmp)
    #decoded_imgs = decoder.predict(encoded_imgs)
    
    #SDAE needs new set of training examples with encoded rep for the next stack
    X_train_tmp = encoder.predict(X_train_tmp)
    print(X_train_tmp.shape)
    
    encoders.append(encoder)
    print(encoders)


Training the layer 1: Input 784 -> Output 676
compiling...
fiting.pretraining for just few epochs..
Epoch 1/1
60000/60000 [==============================] - 5s - loss: 0.3089     
(60000, 676)
Training the layer 2: Input 676 -> Output 484
compiling...
fiting.pretraining for just few epochs..
Epoch 1/1
60000/60000 [==============================] - 4s - loss: 0.5381     
(60000, 484)
[<keras.engine.training.Model object at 0x7fa6af602748>, <keras.engine.training.Model object at 0x7fa6abdbe7b8>]
Training the layer 3: Input 484 -> Output 324
compiling...
fiting.pretraining for just few epochs..
Epoch 1/1
60000/60000 [==============================] - 2s - loss: 0.5389     
(60000, 324)
[<keras.engine.training.Model object at 0x7fa6af602748>, <keras.engine.training.Model object at 0x7fa6abdbe7b8>, <keras.engine.training.Model object at 0x7fa6a7622710>]
Training the layer 4: Input 324 -> Output 144
compiling...
fiting.pretraining for just few epochs..
Epoch 1/1
60000/60000 [================

In [5]:
print(encoders)

[<keras.engine.training.Model object at 0x7fa6af602748>, <keras.engine.training.Model object at 0x7fa6abdbe7b8>, <keras.engine.training.Model object at 0x7fa6a7622710>, <keras.engine.training.Model object at 0x7fa6aea8b320>, <keras.engine.training.Model object at 0x7fa6a76d1518>, <keras.engine.training.Model object at 0x7fa6a76bba20>]


In [6]:
for encoder in encoders:
    #model.add(encoder)
    #x=model.get_weights()
    print(encoder.layers[0])

    #print(encoder.layers[0].get_config())
    print(len(encoder.layers[0].get_weights()))
    
    print(encoder.layers[1])

    #print(encoder.layers[1].get_config())
    print(len(encoder.layers[1].get_weights()))


    ##print(encoder.layers[2]) ---> List Index out of range
    #top_model.layer[2].set_weights(encoder.layers[0].get_weights())
#model.summary()
#print(type(x))
#print(len(x))

0
2
0
2
0
2
0
2
0
2
0
2


In [7]:

print (len(nb_hidden_layers))
print (nb_hidden_layers[0])
print (nb_hidden_layers[1])
print (nb_hidden_layers[2])

print (nb_hidden_layers[-1])
print (nb_hidden_layers[3])

##print (nb_hidden_layers[4])---> List out of range



7
784
676
484
35
324


In [8]:
# Build the top model to Dimension reduction
input_img = Input(shape=(nb_hidden_layers[0],))
top_encoded = Dense(nb_hidden_layers[1], activation='relu')(input_img)
for i in range(len(nb_hidden_layers) - 2) :

    top_encoded = Dense(nb_hidden_layers[i+2], activation='relu')(top_encoded)
    print('i is ',i)
     
#top_encoded = Dense(nb_hidden_layers[3], activation='sigmoid')(top_encoded)

# add the below line classify
top_decoded = Dense(nb_hidden_layers[0], activation='sigmoid')(top_encoded)
top_autoencoder = Model(input=input_img, output=top_decoded)


i is  0
i is  1
i is  2
i is  3
i is  4


In [9]:
# this model maps an input to its encoded representation
top_encoder_only = Model(input=input_img, output=top_encoded)

In [10]:
print(top_autoencoder.layers[0])
print(len(top_autoencoder.layers[0].get_weights()))

print(top_autoencoder.layers[1])
print(len(top_autoencoder.layers[1].get_weights()))

print(top_autoencoder.layers[2])
print(len(top_autoencoder.layers[2].get_weights()))

print(top_autoencoder.layers[3])
print(len(top_autoencoder.layers[3].get_weights()))

print(top_autoencoder.layers[4])
print(len(top_autoencoder.layers[4].get_weights()))

###index out of range
print(top_autoencoder.layers[6])
print(len(top_autoencoder.layers[6].get_weights()))

print(top_encoder_only)

0
2
2
2
2
2


In [11]:
i=1
for encoder in encoders:
    print('i is ',i)
    # stack the first models encoded layer and set weights
    print(len(encoder.layers[1].get_weights()))
    top_autoencoder.layers[i].set_weights(encoder.layers[1].get_weights())
    print(len(top_autoencoder.layers[i].get_weights()))
    i+=1
    #if i == len(nb_hidden_layers)+1:
    #    break


i is  1
2
2
i is  2
2
2
i is  3
2
2
i is  4
2
2
i is  5
2
2
i is  6
2
2


In [12]:
from sklearn.cross_validation import train_test_split

s_train, s_valid = train_test_split(X_train, test_size=0.3)
v_train, v_valid = train_test_split(Y_train, test_size=0.3)

In [13]:
print(top_autoencoder)

top_autoencoder.compile(optimizer=Adam(lr=1e-3), loss='categorical_crossentropy',metrics=['accuracy'])
score = top_autoencoder.evaluate(X_test, X_test, verbose=0)
print('Test score before fine turning:', score[0])
print('Test accuracy before fine turning:', score[1])
top_autoencoder.fit(s_train, s_train, batch_size=batch_size, nb_epoch=nb_epoch,
                    validation_data=(s_valid, s_valid))
score = top_autoencoder.evaluate(X_test, X_test, verbose=0)
print('Test score after fine turning:', score[0])
print('Test accuracy after fine turning:', score[1])

Test score before fine turning: 692.301820703
Test accuracy before fine turning: 0.0008
Train on 42000 samples, validate on 18000 samples
Epoch 1/50
42000/42000 [==============================] - 5s - loss: 587.9797 - acc: 0.0027 - val_loss: 558.0108 - val_acc: 0.0021
Epoch 2/50
42000/42000 [==============================] - 5s - loss: 549.0256 - acc: 0.0017 - val_loss: 542.2636 - val_acc: 0.0012
Epoch 3/50
42000/42000 [==============================] - 5s - loss: 539.6647 - acc: 0.0016 - val_loss: 539.3575 - val_acc: 0.0011
Epoch 4/50
42000/42000 [==============================] - 5s - loss: 534.6969 - acc: 0.0015 - val_loss: 531.7510 - val_acc: 8.8889e-04
Epoch 5/50
42000/42000 [==============================] - 5s - loss: 531.4921 - acc: 0.0015 - val_loss: 529.8543 - val_acc: 0.0015
Epoch 6/50
42000/42000 [==============================] - 5s - loss: 529.2668 - acc: 0.0015 - val_loss: 528.3006 - val_acc: 1.0000e-03
Epoch 7/50
42000/42000 [==============================] - 5s - loss:

In [14]:
train_encoded_imgs = top_encoder_only.predict(X_train,batch_size=batch_size, verbose=1)

59904/60000 [============================>.] - ETA: 0s

In [15]:
print(train_encoded_imgs.shape)

(60000, 35)


In [16]:
test_encoded_imgs = top_encoder_only.predict(X_test,batch_size=batch_size, verbose=1)

 9984/10000 [============================>.] - ETA: 0s

In [17]:
print(test_encoded_imgs.shape)

(10000, 35)


In [18]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier()
clf = clf.fit(train_encoded_imgs, Y_train)

In [19]:
y_pred = clf.predict(test_encoded_imgs)

In [20]:
num=len(test_encoded_imgs)
r=0
w=0
wrngclassidx=[]
#y_test = np.ravel(y_test)
for i in range(num):
        #print ('y_pred ',y_pred[i])
        #print ('labels ',labels[i])
        #without the use of all() returns error truth value of an array with more than one element is ambiguous
        #if y_pred[i].all() == labels[i].all():
        if np.array_equal(y_pred[i],Y_test[i]):
            r+=1
        else:
            w+=1
            wrngclassidx.append(i)
print ("tested ",  num, "digits")
print ("correct: ", r, "wrong: ", w, "error rate: ", float(w)*100/(r+w), "%")
print ("got correctly ", float(r)*100/(r+w), "%")
print (len(wrngclassidx))

tested  10000 digits
correct:  9476 wrong:  524 error rate:  5.24 %
got correctly  94.76 %
524


In [ ]:
Testing....

In [19]:
input_img1 = Input(shape=(784,))
# "encoded" is the encoded representation of the input
encoded1 = Dense(600, activation='sigmoid')(input_img1) #26X26
decoded1 = Dense(784, activation='sigmoid')(encoded1)
# this model maps an input to its reconstruction
autoencoder1 = Model(input=input_img1, output=decoded1)
autoencoder1.compile(optimizer=Adam(lr=1e-3), loss='categorical_crossentropy',metrics=['accuracy'])
autoencoder1.fit(X_train_tmp, X_train_tmp,
                nb_epoch=nb_epoch,
                batch_size=batch_size,
                shuffle=True)
                #validation_data=(s_valid_noisy, s_valid),
                #callbacks=callbacks)
print(autoencoder1.layers[-1])
# Store trainined weight and update training data
encoders.append(autoencoder1.layers[-1])
print(encoders.append)
X_train_tmp = autoencoder1.predict(X_train_tmp)
print(X_train_tmp.shape)
print(X_train_tmp)


Epoch 1/1
60000/60000 [==============================] - 5s - loss: 567.4929 - acc: 0.0062     
<built-in method append of list object at 0x7f4a769c60c8>
(60000, 784)
[[  2.08194950e-04   1.61045158e-04   1.91865693e-04 ...,   4.11617657e-04
    2.06225828e-04   2.25820302e-04]
 [  8.34012681e-05   1.15721450e-04   6.54617252e-05 ...,   1.00052559e-04
    1.18069322e-04   1.19686571e-04]
 [  1.55348040e-03   2.11423798e-03   2.02637399e-03 ...,   1.87618355e-03
    1.72902329e-03   1.79942453e-03]
 ..., 
 [  2.50088226e-04   3.11022683e-04   2.65567418e-04 ...,   5.46693511e-04
    3.07242037e-04   3.17488448e-04]
 [  1.88292644e-04   2.04064956e-04   9.50054527e-05 ...,   2.04343232e-04
    2.87173170e-04   2.82932306e-04]
 [  3.08272341e-04   3.46488523e-04   2.56666623e-04 ...,   5.05295815e-04
    2.51874444e-04   3.89833644e-04]]
